In [3]:
import pandas as pd
import numpy as np
import ast
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [4]:
plt.rcParams['font.family'] = 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

# 데이터 확인

In [5]:
port = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/portfolio.csv",index_col=0)
port.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/portfolio.csv'

In [ ]:
profile = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/profile.csv",index_col=0)
profile.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [ ]:
trans = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/transcript.csv",index_col=0)
trans.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [ ]:
port.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


In [ ]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 796.9+ KB


In [ ]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.7+ MB


In [ ]:
profile.isnull().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

# 전처리

In [ ]:
trans['value'] = trans['value'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
trans['amount'] = trans['value'].apply(lambda x: x.get('amount') if isinstance(x, dict) else None)
trans['amount'] = trans['amount'].astype(float)
trans['offer_id'] = trans['value'].apply(lambda x: x.get('offer id') or x.get('offer_id') if isinstance(x, dict) else None)

In [ ]:
income_median = profile['income'].median()
profile['income'].fillna(income_median, inplace=True)
profile['income']

0         64000.0
1        112000.0
2         64000.0
3        100000.0
4         64000.0
           ...   
16995     54000.0
16996     72000.0
16997     73000.0
16998     50000.0
16999     82000.0
Name: income, Length: 17000, dtype: float64

In [ ]:
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile['became_member_on'] = profile['became_member_on'].dt.date

In [ ]:
trans.drop(columns=['value'], axis=1, inplace=True)
port.drop(columns=['channels'], axis=1, inplace=True)

# 거래 기록이 있는 고객 대상

In [ ]:
trans = trans[trans['event'] == 'transaction']

In [ ]:
trans_df = pd.merge(trans, profile, how = 'left', left_on = 'person', right_on = 'id')
trans_df

,person,event,time,amount,offer_id,gender,age,id,became_member_on,income
0,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83,None,F,20,02c083884c7d45b39cc68e1314fec56c,2016-07-11,30000.0
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56,None,M,42,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,2016-01-17,96000.0
2,54890f68699049c2a04d415abc25e717,transaction,0,13.23,None,M,36,54890f68699049c2a04d415abc25e717,2017-12-28,56000.0
3,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51,None,F,55,b2f1cd155b864803ad8334cdf13c4bd2,2017-10-16,94000.0
4,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97,None,F,39,fe97aa22dd3e48c8b143116a8403dd52,2017-12-17,67000.0
...,...,...,...,...,...,...,...,...,...,...
138948,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,None,M,66,b3a1272bc9904337b331bf348c3e8c17,2018-01-01,47000.0
138949,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,None,M,52,68213b08d99a4ae1b0dcb72aebd9aa35,2018-04-08,62000.0
138950,a00058cf10334a308c68e7631c529907,transaction,714,3.61,None,F,63,a00058cf10334a308c68e7631c529907,2013-09-22,52000.0
138951,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,None,M,57,76ddbd6576844afe811f1a3c0fbb5bec,2016-07-09,40000.0


In [ ]:
trans_df.drop(columns=['offer_id'], axis=1, inplace=True)

In [ ]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138953 entries, 0 to 138952
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   person            138953 non-null  object 
 1   event             138953 non-null  object 
 2   time              138953 non-null  int64  
 3   amount            138953 non-null  float64
 4   gender            123957 non-null  object 
 5   age               138953 non-null  int64  
 6   id                138953 non-null  object 
 7   became_member_on  138953 non-null  object 
 8   income            138953 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 9.5+ MB


In [ ]:
# CSV 불러오기
df = pd.read_csv("마케팅 스타벅스 통합데이터.csv")

# 날짜 변환 (time은 일(day) 단위라고 가정, 기준 날짜 설정)
df['event_date'] = pd.to_timedelta(df['time'], unit='D') + pd.Timestamp('2018-01-01')

# 구매 데이터만 필터링
tx_df = df[df['event'] == 'transaction']

# 오늘 날짜(분석 기준일) → 데이터에서 가장 마지막 거래일 + 1일
snapshot_date = tx_df['event_date'].max() + pd.Timedelta(days=1)

# 고객별 RFM 계산
rfm = tx_df.groupby('person').agg({
    'event_date': lambda x: (snapshot_date - x.max()).days,  # Recency
    'person': 'count',                                      # Frequency
    'amount': 'sum'                                         # Monetary
}).rename(columns={
    'event_date': 'Recency',
    'person': 'Frequency',
    'amount': 'Monetary'
}).reset_index()

# RFM 점수 (1~5)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])  # 낮을수록 점수 높음
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

# RFM 세그먼트
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score','F_Score','M_Score']].sum(axis=1).astype(int)

# 상위 몇 행 보기
print(rfm.head())

# RFM 등급 분류 예시
def rfm_level(row):
    if row['RFM_Score'] >= 12:
        return 'High Value'
    elif row['RFM_Score'] >= 8:
        return 'Medium Value'
    else:
        return 'Low Value'

rfm['Segment'] = rfm.apply(rfm_level, axis=1)

print(rfm['Segment'].value_counts())

In [ ]:
plt.figure(figsize=(6,5))
sns.countplot(data=rfm, x='Segment', order=rfm['Segment'].value_counts().index, palette='Set2')
plt.title('세그먼트별 고객 수')
plt.xlabel('세그먼트')
plt.ylabel('고객 수')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=rfm, x='Recency', y='Monetary', hue='Segment', palette='Set2')
plt.title('Recency vs Monetary - 세그먼트별 분포')
plt.xlabel('Recency(최근 구매 이후 경과일)')
plt.ylabel('Monetary(총 구매 금액)')
plt.legend(title='세그먼트')
plt.show()
